In [1]:
!pip install pydantic

## Data Validation with `pydantic`

In [2]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    age: int
    city: str

person=Person(name="Dhruv", age=19, city="Bangalore")
print(person)

name='Dhruv' age=19 city='Bangalore'


In [3]:
type(person)

__main__.Person

What's the difference between pydantic (inheriting from BaseModel) vs just using a normal decorator class?

In [5]:
# normal class
from dataclasses import dataclass

@dataclass
class Person1():
    name: str
    age: int
    city: str

person1=Person1(name="Dhruv", age=19, city="Bangalore")
print(person1)

Person1(name='Dhruv', age=19, city='Bangalore')


This prints normally, but compare the case when I put some random number instead of a string in `city`

In [6]:
# normal class
dataclass_person=Person1(name="Dhruv", age=19, city=21)
print(dataclass_person)

# pydantic class inherited from BaseModel
pydantic_class = Person(name="Dhruv", age=19, city=21)
print(pydantic_class)

Person1(name='Dhruv', age=19, city=21)


ValidationError: 1 validation error for Person
city
  Input should be a valid string [type=string_type, input_value=21, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

You can see the immediate difference when we use this because there is a validation error as the data entered doesnt fit the datatype schema explicitly defined by us earlier.

## Using Optional Fields

add optional fields using python's optional type from typing

In [22]:
from typing import Optional

class Employee(BaseModel):
    id: int
    name: str
    department: str
    salary: Optional[float] = None # set an optional, along with the default value while using this
    active_status: Optional[bool] = True

Note: 
- Optional[type]: Indicates that the field can be `None`
- Required fields must still be provided
- Pydantic will validate the type **even for optional fields, when values are provided only**

Also, one of the fields can also be a list 

In [9]:
from pydantic import BaseModel
from typing import List, Optional

class Classroom(BaseModel):
    room_number: str
    students: List[str] # list of strings
    capacity: Optional[int] = 2
    
classroom = Classroom(
    room_number="A-507",
    students=['Dhruv', 'Prakhar']
)

print(classroom)

room_number='A-507' students=['Dhruv', 'Prakhar'] capacity=2


## Using try and except clauses to bypass validation errors

In [11]:
try:
    trial_val = Classroom(room_number='A-506', students=["Shubhu",2], capacity=2)
except ValueError as e:
    print(e)

1 validation error for Classroom
students.1
  Input should be a valid string [type=string_type, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


## Nested Models and Creating Complex Data Structures
Nested models are models in which the datatype of some variable in one class is actually inheriting from some other class, eg:

In [12]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    pin_code: int
    
class Customer(BaseModel):
    customer_id: int
    name: str
    address: Address # this is why it's a nested model
    
# creating a customer with a nested address

customer = Customer(
    customer_id=1,
    name='Dhruv',
    address=Address(
        street='King\'s Cross Station',
        city='London',
        pin_code=21
    )
)

print(customer)

customer_id=1 name='Dhruv' address=Address(street="King's Cross Station", city='London', pin_code=21)


We can also use a dictionary to do this since it already knows what Class to inherit from, so pass in args directly

In [16]:
customer = Customer(
    customer_id=1,
    name='Dhruv',
    address={"street":'King\'s Cross Station', "city":'London', "pin_code":21}
)

print(customer)

customer_id=1 name='Dhruv' address=Address(street="King's Cross Station", city='London', pin_code=21)


## Pydantic Fields

There is a `Field` function in pydantic that enhances model fields beyond just basic type hints, it allows to specify validation rules, default values, aliases, etc. 

In [21]:
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str=Field(min_length=2, max_length=50)
    price: float=Field(gt=0, lt=100) # price should be greater than 0, less than 100
    qty:int=Field(gt=0, lt=10)
    
# valid instance
item1 = Item(name='Pencils', price=35, qty=5)
print(item1)


# invalid instance
item2 = Item(name='e', price=1000, qty=100)

name='Pencils' price=35.0 qty=5


ValidationError: 3 validation errors for Item
name
  String should have at least 2 characters [type=string_too_short, input_value='e', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
price
  Input should be less than 100 [type=less_than, input_value=1000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than
qty
  Input should be less than 10 [type=less_than, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than

## Using Descriptions (helpful while looking at the schema)

When we give the schema, atleast write enough metadata such that the developer or third party is able to understand what datatypes and stuff there is in the class.

In [25]:
from pydantic import BaseModel, Field

class User(BaseModel):
    username: str=Field(..., description="Unique Username")
    age: int=Field(default=18, description="User age that defaults to 18")
    email: str=Field(default_factory = lambda: "user@example.com", description="Default email address is: user@example.com")

# printing the schema
print(User.model_json_schema())

{'properties': {'username': {'description': 'Unique Username', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age that defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Default email address is: user@example.com', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
